# Labeling the [Lexical Relationship](https://huggingface.co/datasets/relbert/lexical_relation_classification) dataset using Autolabel

This is a multi-class classification task where the input are two english words and we have to correctly classify the lexical relationship between them using one of 5 labels. 

## Install Autolabel
Plus, setup your OpenAI API key, since we'll be using `gpt-3.5-turbo` as our LLM for labeling.

In [12]:
!pip install 'refuel-autolabel[openai]'


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip


In [1]:
import os

# provide your own OpenAI API key here
os.environ["OPENAI_API_KEY"] = "sk-"


## Download the dataset

This dataset is available to install via Autolabel.

In [2]:
from autolabel import get_data

get_data("lexical_relation")

This downloads two datasets:
* `test.csv`: This is the larger dataset we are trying to label using LLMs
* `seed.csv`: This is a small dataset where we already have human-provided labels

## Start the labeling process!

Labeling with Autolabel is a 3-step process:
* First, we specify a labeling configuration (see `config.json` below)
* Next, we do a dry-run on our dataset using the LLM specified in `config.json` by running `agent.plan`
* Finally, we run the labeling with `agent.run`

### First labeling run

In [3]:
import json

from autolabel import LabelingAgent

In [4]:
# load the config
with open("config_lexical_relation.json") as f:
     config = json.load(f)

Let's review the configuration file below. You'll notice the following useful keys:
* `task_type`: `classification` (since it's a classification task)
* `model`: `{'provider': 'openai', 'name': 'gpt-3.5-turbo'}` (use a specific OpenAI model)
* `prompt.task_guidelines`: `'You are an expert at understanding bank customers support complaints and queries...` (how we describe the task to the LLM)
* `prompt.labels`: `['age_limit', 'apple_pay_or_google_pay', 'atm_support', ...]` (the full list of labels to choose from)
* `prompt.few_shot_num`: 10 (how many labeled examples to provide to the LLM)

In [5]:
config

{'task_name': 'LexicalRelationClassification',
 'task_type': 'classification',
 'dataset': {'label_column': 'label', 'delimiter': ','},
 'model': {'provider': 'openai', 'name': 'gpt-3.5-turbo'},
 'prompt': {'task_guidelines': 'You are an expert at understanding lexical relationships.\n Given two words A and B, classify the relationship between them into one of: {labels}. RANDOM refers to an arbitrary relationship. SYN means that the words are synonyms. ANT means that the two words are antonyms. HYPER means that the two words are hypernyms. PART_OF means that the second word is a subset of the first word.',
  'output_guidelines': 'You will answer with just the the correct output label and nothing else.',
  'labels': ['RANDOM', 'SYN', 'ANT', 'HYPER', 'PART_OF'],
  'few_shot_examples': 'seed.csv',
  'few_shot_selection': 'semantic_similarity',
  'few_shot_num': 10,
  'example_template': 'Input: {example}\nOutput: {label}'}}

In [6]:
# create an agent for labeling
agent = LabelingAgent(config=config)

In [7]:
# dry-run -- this tells us how much this will cost and shows an example prompt
from autolabel import AutolabelDataset

ds = AutolabelDataset("data/lexical_relation/test.csv", config=config)
agent.plan(ds)

Output()

┌──────────────────────────┬─────────┐
│ Total Estimated Cost     │ $4.8362 │
│ Number of Examples       │ 2000    │
│ Average cost per example │ $0.0024 │
└──────────────────────────┴─────────┘

───────────────────────────────────────────────── Prompt Example ──────────────────────────────────────────────────

You are an expert at understanding lexical relationships.
 Given two words A and B, classify the relationship between them into one of: RANDOM
SYN
ANT
HYPER
PART_OF. RANDOM refers to an arbitrary relationship. SYN means that the words are synonyms. ANT means that the two 
words are antonyms. HYPER means that the two words are hypernyms. PART_OF means that the second word is a subset of
the first word.

You will answer with just the the correct output label and nothing else.

Some examples with their output answers are provided below:

Input: A: bowl
B: dish
Output: HYPER

Input: A: ignorance
B: dish
Output: RANDOM

Input: A: chicken
B: food
Output: HYPER

Input: A: america
B: country
Output: HYPER

Input: A: apartment
B: play
Output: RANDOM

Input: A: beef
B: team
Output: RANDOM

Input: A: captain
B: team
Output: PART_OF

Input: A: actress
B: person
Output: HYPER

Input: A: monitor
B: television
Output: PART_OF

Input: A: hit
B: food
Output: RANDOM

Now I want you to label the following example:
Input: A: company
B: dish
Output:

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [8]:
# now, do the actual labeling
ds = agent.run(ds, max_items=100)

2023-09-29 00:03:26 autolabel.labeler INFO: Task run already exists.


There is an existing task with following details: id='1029336087' created_at=datetime.datetime(2023, 9, 29, 0, 2, 
24, 415913) task_id='7071645cb8e8bf65daaf1770b1b51d08' dataset_id='a520beb48d484eec1cfb9d39d0152ece' 
current_index=67 output_file='LexicalRelationClassification_labeled.csv' status=<TaskStatus.ACTIVE: 'active'> 
error=None metrics=None

Evaluating the existing task...

┏━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ accuracy ┃ support ┃ completion_rate ┃
┡━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ 0.0      │ 66      │ 0.0             │
└──────────┴─────────┴─────────────────┘

67 examples labeled so far.

...

Deleted the existing task and starting a new one...

Output()

classification_report:
              precision    recall  f1-score   support

         ANT       0.78      0.78      0.78         9
       HYPER       0.67      0.29      0.40         7
     PART_OF       0.18      0.67      0.29         3
      RANDOM       0.96      0.89      0.92        75
         SYN       0.71      0.83      0.77         6

    accuracy                           0.83       100
   macro avg       0.66      0.69      0.63       100
weighted avg       0.88      0.83      0.85       100

Actual Cost: 0.0421

┏━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ accuracy ┃ support ┃ completion_rate ┃
┡━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ 0.83     │ 100     │ 1.0             │
└──────────┴─────────┴─────────────────┘